<a href="https://colab.research.google.com/github/MustafaEisaTech/Learning_TF/blob/main/NN_with_sequential_and_functional_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#let's start by importing the models we need
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
physical_device = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_device[0], True)

In [32]:
# Now we need to load the data and preprocess it
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 784).astype('float32') / 255.0 #Reshape into a vector to input it into the NN
x_test = x_test.reshape(-1, 784).astype('float32') / 255.0
print(x_train.shape)

(60000, 784)


In [11]:
# We are going to create a model using Keras's Sequntial API
#it maps one input to one output
# create a model with a list inside that conatins all the layers
#use the dense method and define the nodes and set the activation functions(layer.Dense())
#Usually don't use an activation for the last layer
model = keras.Sequential(
    [
      layers.Dense(512, activation = 'relu'),
      layers.Dense(256, activation = 'relu' ),
      layers.Dense(10)
    ]
)

#After making the architecture, we can use the compile method for the training part
# we can specify a loss function keras.losses.our choice and put it into a variable called loss
# If we are not going to use a softmax in the last layer we usually need to set the from_logits parameter to true to send it through a softmax first
# In the tutorial Sparse Categorical Cross entropy was used to use the label as an integer. Example if it was image of the digit 3, then it would sparse the value to 3 as a label
# pick an optimizer using keras.optimizer.choose ur optimizer, the author used Adam and this method takes a parametr lr which is learning rate
# specify the metrics = ["accuracy"]
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    optimizer = keras.optimizers.Adam(learning_rate = 0.001),
    metrics = ['accuracy']
)


#Model fitting
#we want to send our training sets as parameter and specify the batch size and the epochs, we can set a value to verbose to print the process
model.fit(x_train, y_train, batch_size = 32, epochs = 5, verbose = 2)
#Model Evaluation
#takes the test sets and the same parameters for the fit method expect the epochs
model.evaluate(x_test, y_test, batch_size = 32, verbose = 2)




Epoch 1/5
1875/1875 - 4s - 2ms/step - accuracy: 0.9434 - loss: 0.1867
Epoch 2/5
1875/1875 - 6s - 3ms/step - accuracy: 0.9745 - loss: 0.0803
Epoch 3/5
1875/1875 - 3s - 2ms/step - accuracy: 0.9818 - loss: 0.0561
Epoch 4/5
1875/1875 - 3s - 2ms/step - accuracy: 0.9868 - loss: 0.0411
Epoch 5/5
1875/1875 - 6s - 3ms/step - accuracy: 0.9893 - loss: 0.0339
313/313 - 1s - 3ms/step - accuracy: 0.9767 - loss: 0.0847


[0.08472231775522232, 0.9767000079154968]

In [47]:
# there are different ways u can make the architecture in, u can use this :
model = keras.Sequential()
model.add(keras.Input(shape = (784, )))
model.add(layers.Dense(512, activation = 'relu'))
model.add(layers.Dense(256, activation = 'relu', name = '256_layer'))
model.add(layers.Dense(10))

#model = keras.Model(inputs = inputs, outputs = [model.layer[-2].output])
#model = keras.Model(inputs = model.inputs, outputs = [model.get_layer('256_layer').output])
model = keras.Model(inputs = model.inputs, outputs = [layer.output for layer in model.layers])
#feature = model.predict(x_train)
#print(feature.shape)

features = model.predict(x_train)
for feature in features:
  print(feature.shape)

import sys
sys.exit()

1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
(60000, 512)
(60000, 256)
(60000, 10)


SystemExit: 

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [48]:
#The Functional API, the thing we use when the Sequential API is not convinent
#it can handle multiple inputs and multiple outputs
#we first define the shape of the inputs
# it takes the layer before it
inputs = keras.Input(shape = (784, ))
x = layers.Dense(512, activation = 'relu', name = 'first_layer')(inputs)
x = layers.Dense(256, activation = 'relu', name = 'second_layer')(x)
outputs = layers.Dense(10, activation = 'softmax')(x)
model = keras.Model(inputs = inputs, outputs = outputs)
print(model.summary())

model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits = False),
    optimizer = keras.optimizers.Adam(learning_rate = 0.001),
    metrics = ['accuracy']
)

model.fit(x_train, y_train, batch_size=32, epochs = 10, verbose = 2)
model.evaluate(x_test, y_test, batch_size = 32, verbose = 2)

Model: "functional_48"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_36 (InputLayer)          │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ first_layer (Dense)                  │ (None, 512)                 │         401,920 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ second_layer (Dense)                 │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_90 (Dense)                     │ (None, 10)                  │           2,570 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 535,818 (2.04 MB)

 Trainable params: 535,818 (2.04 MB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/10
1875/1875 - 5s - 3ms/step - accuracy: 0.9433 - loss: 0.1863
Epoch 2/10
1875/1875 - 4s - 2ms/step - accuracy: 0.9749 - loss: 0.0787
Epoch 3/10
1875/1875 - 5s - 3ms/step - accuracy: 0.9828 - loss: 0.0543
Epoch 4/10
1875/1875 - 4s - 2ms/step - accuracy: 0.9868 - loss: 0.0405
Epoch 5/10
1875/1875 - 3s - 1ms/step - accuracy: 0.9888 - loss: 0.0336
Epoch 6/10
1875/1875 - 5s - 3ms/step - accuracy: 0.9912 - loss: 0.0268
Epoch 7/10
1875/1875 - 5s - 3ms/step - accuracy: 0.9928 - loss: 0.0235
Epoch 8/10
1875/1875 - 5s - 3ms/step - accuracy: 0.9931 - loss: 0.0211
Epoch 9/10
1875/1875 - 3s - 1ms/step - accuracy: 0.9941 - loss: 0.0174
Epoch 10/10
1875/1875 - 4s - 2ms/step - accuracy: 0.9947 - loss: 0.0166
313/313 - 1s - 3ms/step - accuracy: 0.9795 - loss: 0.0972


[0.0972265973687172, 0.9794999957084656]

In [52]:
inputs = keras.Input(shape = (784, ))
l1 = layers.Dense(512, activation = 'sigmoid')(inputs)
l2 = layers.Dense(256, activation = 'sigmoid')(l1)
outputs = layers.Dense(10, activation = 'softmax')(l2)

model = keras.Model(inputs = inputs, outputs = outputs)

model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer = keras.optimizers.SGD(learning_rate=0.001),
    metrics = ['accuracy']
)

model.fit(x_train, y_train, batch_size=64, epochs = 100, verbose = 2)
model.evaluate(x_test, y_test, batch_size=64, verbose = 2 )

Epoch 1/100
938/938 - 3s - 3ms/step - accuracy: 0.1284 - loss: 2.3001
Epoch 2/100
938/938 - 2s - 2ms/step - accuracy: 0.1563 - loss: 2.2746
Epoch 3/100
938/938 - 1s - 1ms/step - accuracy: 0.2239 - loss: 2.2594
Epoch 4/100
938/938 - 3s - 3ms/step - accuracy: 0.2899 - loss: 2.2437
Epoch 5/100
938/938 - 3s - 3ms/step - accuracy: 0.3416 - loss: 2.2275
Epoch 6/100
938/938 - 1s - 1ms/step - accuracy: 0.4252 - loss: 2.2107
Epoch 7/100
938/938 - 1s - 1ms/step - accuracy: 0.4603 - loss: 2.1929
Epoch 8/100
938/938 - 2s - 2ms/step - accuracy: 0.4958 - loss: 2.1741
Epoch 9/100
938/938 - 2s - 2ms/step - accuracy: 0.5381 - loss: 2.1542
Epoch 10/100
938/938 - 1s - 1ms/step - accuracy: 0.5720 - loss: 2.1326
Epoch 11/100
938/938 - 3s - 3ms/step - accuracy: 0.5827 - loss: 2.1096
Epoch 12/100
938/938 - 3s - 3ms/step - accuracy: 0.6018 - loss: 2.0848
Epoch 13/100
938/938 - 1s - 1ms/step - accuracy: 0.6163 - loss: 2.0576
Epoch 14/100
938/938 - 3s - 3ms/step - accuracy: 0.6308 - loss: 2.0284
Epoch 15/100
93

[0.4483986794948578, 0.8808000087738037]